In [18]:
import gc
import os
import functools
import tqdm
import shutil
import requests
import pandas as pd
import numpy as np

from pyunpack import Archive
from bs4 import BeautifulSoup
from pathlib import Path
from multiprocessing import Pool
from dbfread import DBF
from typing import Union

import re
import time

In [19]:
def dbf2df(filepath: str, encoding:str) -> pd.DataFrame:
    """
    Reads .dbf from given filepath into dataframe and returns df
    """
    dbf = DBF(filepath, encoding=encoding)
    df = pd.DataFrame(iter(dbf)) 
    # if there is no 'DT' column with date
    if 'DT' not in df.columns:
        # get date from folder name
        try:
            date = str(filepath).split('/')[-2].split('-')[1]
        # path with ordinary slash if working on Linux 
        except IndexError:
            date = str(filepath).split('/')[-2].split('-')[1]
        df['DT'] = pd.to_datetime(date)
    return df
    

def get_filepaths(filepath:str):
    """достает названия конечных файлов"""
    folders = [filelist[0] for filelist in os.walk(filepath)][1:]
    files = []
    for folder in folders:
        for file in os.listdir(folder):
            files.append(Path(folder)/file)
    return files


def get_bank_names(filepath:str, 
                  form_number:int, 
                  encoding:str='cp866') -> pd.DataFrame:
    """
    Collects all bank names and register numbers from given form 
    (from files with 'N1' for form 101 and 'NP1' for form 102). Returns 
    Pandas DataFrame with 2 columns: register number (REGN) and name 
    of the bank.
    
    Parameters
    ----------
    path: str
            Path to folder with folders with downloaded and unzipped files
    encoding:str, default 'cp866'
            Encoding to open .dbf files from CBR. Recommended value is 'cp866'
    form_number: int, 101 or 102
            Whether to collect bank names from files with 101 or 102 form
    """
    # collect all file paths from folders in given path
    files = get_filepaths(filepath)
            
    # different files with names for different forms
    search = {101:'N1', 102:'NP1'}
    # find all files with matching pattern in file names
    names = list(filter(lambda x: \
                        search[form_number] in str(x), files))
    
    # read all these files and merge them into one file
    df = functools.reduce(lambda a,b: \
                          pd.concat((a.reset_index(drop=True), 
                                     b.reset_index(drop=True))),
                          [dbf2df(x, encoding=encoding) for x in names])
    
    # integer codes ('REGN') are unique for all banks, but 
    # the same bank names are sometimes written in different ways
    df.drop_duplicates(subset='REGN', inplace=True)
    return df[['REGN', 'NAME_B']].reset_index(drop=True)


In [20]:
form101=get_bank_names('/Users/roman142001/Documents/diplom/data/', 
                  101, 
                  'cp866')
form101.head(10)

,REGN,NAME_B
0,1,ЗАО ЮниКредит Банк
1,1000,ОАО Банк ВТБ
2,1002,"ОАО Комсоцбанк ""Бумеранг"""
3,1003,"ООО ""ЛЕНОБЛБАНК"""
4,1006,"ОАО ""Башпромбанк"""
5,1016,"""Объединенный банк Республики"" (ООО)"
6,1025,"ООО КБ ""Эл банк"""
7,1027,ОАО ВКАБАНК
8,1035,"ОАО ККБ ""КРЕДИТБАНК"""
9,1037,"ООО КБ ""Майма"""


In [22]:
form101.to_csv('fom101names')

In [21]:
form102=get_bank_names('/Users/roman142001/Documents/diplom/data/', 
                  102, 
                  'cp866')
form102.head(10)

,REGN,NAME_B
0,1,АО ЮниКредит Банк
1,2,"АО ""КАБ ""Викинг"""
2,21,"ООО ""Примтеркомбанк"""
3,52,"ПАО ""Энергомашбанк"""
4,53,"ПАО Банк ""АЛЕКСАНДРОВСКИЙ"""
5,55,"ПАО МАБ ""Темпбанк"""
6,65,"ООО КБ ""КОЛЬЦО УРАЛА"""
7,67,"АКБ ""Энергобанк"" (ПАО)"
8,77,МОРСКОЙ БАНК (ОАО)
9,85,"ПАО КБ ""САММИТ БАНК"""


In [23]:
form102.to_csv('fom102names')

In [7]:
mismatch = pd.concat([form101, form102]).drop_duplicates(subset='REGN', keep=False)
mismatch
print(f'Количество банков, не попавших в обе формы: {mismatch.shape[0]}')

Количество банков, не попавших в обе формы: 7


ну 7 это не страшно можно будет их и удалить вообще

теперь открываем сами отчеты

In [15]:
def read_form(filepath:str, 
              form_number:int, 
              which_files:str=None, 
              remove_unknown_accs:bool=True,
              to_int:bool=True, 
              encoding:str='cp866') -> pd.DataFrame:
    '''
    Reads and merges all .dbf files for given form and filepath. Returns merded
    dataframe.
    Parameters
    ----------
    filepath: str
            Directory (folder) where are stored .dbf files for form 101 or 102
    form_number: int, 101 or 102 
            Number of CBR form of financial statements (form 101, form 102 etc)
    which_files: str, default None
            Search pattern to look for in file names. For example, by default the 
            function opens and merges all files with 'B1' in filename for 
            form  101 and all files with '_P1' in filenames for 102 form. You can set your 
            own search pattern, but you should be sure, that all files with that pattern
            have the same column names and column order, otherwise function will return
            garbage.
    remove_unknown_accs: bool, default True
            Whether to remove unknown accounts from columns with account number. 
            There are some accounts in form 101 whose meaning I could not find 
            in the Central Bank documents. These accounts are 'ITGAP', '304.1', '408.1', 
            '408.2', '474.1', '1XXXX', '2XXXX', '3XXXX', '4XXXX', '5XXXX', '6XXXX',
            '7XXXX', '8XXXX', '9XXXX'. Removing them allows us to convert column with
            account number from string to integer. This conversion boosts performance
            in dataframe processing and memory management.
    to_int: bool, default True
            Whether to convert column with account numbers to int.
    enconding: str, default 'cp866'
            Encoding to open .dbf files. 'cp866' works well with form 101 and 102
    '''
    
    print('Reading .dbf files from your PC, please wait...')

    files = get_filepaths(filepath)
    # different files with names for different forms
    search = {101:'B1', 102:'_P1'}
    # columns with account numbers
    acc_cols = {101:'NUM_SC', 102:'CODE'}
    # accounts to remove if remove_unknown_accs=True
    remove_accounts = ['ITGAP', '304.1', '408.1', '408.2', '474.1'] + \
                      [str(x)+'XXXX' for x in range(1, 9, 1)]
    # find all files with matching pattern in file names
    if which_files:
        search_str = which_files
    else:
        search_str = search[form_number]

    names = list(filter(lambda x: search_str in str(x), files))
    
    args = ((name, encoding) for name in names)
    print('поехали')
    
    
    #with Pool() as pool:
       # dfs = list(pool.starmap(dbf2df, 
       #           #to show progress bar
              #  tqdm.tqdm(args, total=len(names))))
    df=pd.DataFrame()
    for i in range(len(names)):
        print(i,'/',len(names))
        dfs=dbf2df(names[i],'cp866')
        df=pd.concat([df,dfs])
    

    print('Opened files. Merging them...')


    # delete large list of files from memory
    del dfs
    gc.collect()

    # make date column to datetime index
    df.index = pd.to_datetime(df['DT'])
    df.sort_index(inplace=True)
    df.drop(columns='DT', inplace=True)
    # remove unknown accounts and convert account numbers to int
    if all([remove_unknown_accs, form_number==101]):
        df = df[~df['NUM_SC'].isin(remove_accounts)]
    # convert account numbers column to integer
    if to_int:
        if not remove_unknown_accs and form_number==101:
            raise TypeError(
                """
                You have not removed some very specific accounts
                (remove_unknown_accs=False). This accounts (for 
                instance, 3XXXX) can not be converted to integer.
                """
                )
        df[acc_cols[form_number]] = df[acc_cols[form_number]].astype('int32')

    print('Done.')
    return df

In [9]:
form_101 = read_form('/Users/roman142001/Documents/diplom/data/101', form_number=101)

Reading .dbf files from your PC, please wait...
поехали
0 / 177
1 / 177
2 / 177
3 / 177
4 / 177
5 / 177
6 / 177
7 / 177
8 / 177
9 / 177
10 / 177
11 / 177
12 / 177
13 / 177
14 / 177
15 / 177
16 / 177
17 / 177
18 / 177
19 / 177
20 / 177
21 / 177
22 / 177
23 / 177
24 / 177
25 / 177
26 / 177
27 / 177
28 / 177
29 / 177
30 / 177
31 / 177
32 / 177
33 / 177
34 / 177
35 / 177
36 / 177
37 / 177
38 / 177
39 / 177
40 / 177
41 / 177
42 / 177
43 / 177
44 / 177
45 / 177
46 / 177
47 / 177
48 / 177
49 / 177
50 / 177
51 / 177
52 / 177
53 / 177
54 / 177
55 / 177
56 / 177
57 / 177
58 / 177
59 / 177
60 / 177
61 / 177
62 / 177
63 / 177
64 / 177
65 / 177
66 / 177
67 / 177
68 / 177
69 / 177
70 / 177
71 / 177
72 / 177
73 / 177
74 / 177
75 / 177
76 / 177
77 / 177
78 / 177
79 / 177
80 / 177
81 / 177
82 / 177
83 / 177
84 / 177
85 / 177
86 / 177
87 / 177
88 / 177
89 / 177
90 / 177
91 / 177
92 / 177
93 / 177
94 / 177
95 / 177
96 / 177
97 / 177
98 / 177
99 / 177
100 / 177
101 / 177
102 / 177
103 / 177
104 / 177
105 

In [11]:
form_101.to_hdf('/Users/roman142001/Documents/diplom/dataframe/form101.h5', key='101', mode='w')

/Users/roman142001/opt/anaconda3/lib/python3.8/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: '101'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


In [16]:
form_102 = read_form('/Users/roman142001/Documents/diplom/data/102', form_number=102)

Reading .dbf files from your PC, please wait...
поехали
0 / 61
1 / 61
2 / 61
3 / 61
4 / 61
5 / 61
6 / 61
7 / 61
8 / 61
9 / 61
10 / 61
11 / 61
12 / 61
13 / 61
14 / 61
15 / 61
16 / 61
17 / 61
18 / 61
19 / 61
20 / 61
21 / 61
22 / 61
23 / 61
24 / 61
25 / 61
26 / 61
27 / 61
28 / 61
29 / 61
30 / 61
31 / 61
32 / 61
33 / 61
34 / 61
35 / 61
36 / 61
37 / 61
38 / 61
39 / 61
40 / 61
41 / 61
42 / 61
43 / 61
44 / 61
45 / 61
46 / 61
47 / 61
48 / 61
49 / 61
50 / 61
51 / 61
52 / 61
53 / 61
54 / 61
55 / 61
56 / 61
57 / 61
58 / 61
59 / 61
60 / 61
Opened files. Merging them...
Done.


In [17]:
form_102.to_hdf('/Users/roman142001/Documents/diplom/dataframe/form102.h5', key='102', mode='w')

/Users/roman142001/opt/anaconda3/lib/python3.8/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: '102'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
